In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time

class CompressedAttention(nn.Module):
    """
    Description:
      Реализация механизма сжатого внимания (Compressed Attention) из метода NSA

    Параметры:
        hidden_size (int): Размер скрытого состояния
        block_size (int): Размер блока для сжатия (параметр l в статье)
        stride (int): Шаг между блоками (параметр d в статье)
        num_heads (int): Количество голов внимания
        dropout (float): Вероятность дропаута
    """
    def __init__(self, hidden_size, block_size=32, stride=16, num_heads=4, dropout=0.1):
        super(CompressedAttention, self).__init__()

        self.hidden_size = hidden_size
        self.block_size = block_size
        self.stride = stride
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        # Проекции для запросов, ключей и значений
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)

        # Проекция для выхода
        self.out_proj = nn.Linear(hidden_size, hidden_size)

        # Функция сжатия φ (MLP для сжатия блоков)
        self.block_compressor = nn.Sequential(
            nn.Linear(block_size * self.head_dim, 2 * self.head_dim),
            nn.GELU(),
            nn.Linear(2 * self.head_dim, self.head_dim)
        )

        self.dropout = nn.Dropout(dropout)
        self.scale = self.head_dim ** -0.5

    def _get_blocks(self, x, block_size, stride):
        """
        Description:
          Разбивает последовательность на блоки с заданным размером и шагом

        Аргументы:
            x: тензор формы (batch_size, seq_len, hidden_size)
            block_size: размер блока
            stride: шаг между блоками

        Возвращает:
            blocks: список блоков
            block_indices: список диапазонов индексов для каждого блока
        """
        batch_size, seq_len, hidden_size = x.shape
        blocks = []
        block_indices = []

        # Создаем блоки с перекрытием
        for i in range(0, seq_len - block_size + 1, stride):
            block = x[:, i:i+block_size, :]  # (batch_size, block_size, hidden_size)
            blocks.append(block)
            block_indices.append((i, i+block_size))

        return blocks, block_indices

    def compress_blocks(self, blocks, head_dim):
        """
        Description:
          Сжимает блоки токенов в единые представления с помощью MLP

        Аргументы:
            blocks: список блоков формы (batch_size, block_size, head_dim)
            head_dim: размерность головы внимания

        Возвращает:
            compressed_blocks: тензор формы (batch_size, num_blocks, head_dim)
        """
        batch_size = blocks[0].shape[0]
        num_blocks = len(blocks)

        # Объединяем все блоки в один тензор
        blocks_tensor = torch.cat([block.unsqueeze(1) for block in blocks], dim=1)  # (batch_size, num_blocks, block_size, head_dim)

        # Решейп для передачи в MLP
        reshaped_blocks = blocks_tensor.reshape(batch_size * num_blocks, -1)        # (batch_size * num_blocks, block_size * head_dim)

        # Применяем сжатие (функция φ из статьи)
        compressed = self.block_compressor(reshaped_blocks)                         # (batch_size * num_blocks, head_dim)

        # Приводим к нужной форме
        compressed_blocks = compressed.reshape(batch_size, num_blocks, head_dim)    # (batch_size, num_blocks, head_dim)

        return compressed_blocks

    def forward(self, hidden_states, attention_mask=None, output_attentions=False):
        """
        Description:
          Выполняет сжатое внимание над входной последовательностью

        Аргументы:
            hidden_states: тензор формы (batch_size, seq_len, hidden_size)
            attention_mask: маска внимания
            output_attentions: флаг для вывода матрицы внимания

        Возвращает:
            context_layer: тензор выхода формы (batch_size, seq_len, hidden_size)
            attention_probs (опционально): матрица внимания
        """
        batch_size, seq_len, _ = hidden_states.shape

        # Шаг 1: Проекции запросов, ключей и значений
        q = self.q_proj(hidden_states)  # (batch_size, seq_len, hidden_size)
        k = self.k_proj(hidden_states)  # (batch_size, seq_len, hidden_size)
        v = self.v_proj(hidden_states)  # (batch_size, seq_len, hidden_size)

        # Разделение на головы внимания
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  # (batch_size, num_heads, seq_len, head_dim)

        # Шаг 2: Получение блоков и их сжатие для ключей и значений
        all_compressed_k  = []
        all_compressed_v  = []
        all_block_indices = []

        # Применяем для каждой головы внимания отдельно
        for h in range(self.num_heads):
            head_k = k[:, h]  # (batch_size, seq_len, head_dim)
            head_v = v[:, h]  # (batch_size, seq_len, head_dim)

            # Разбиваем на блоки и получаем их индексы
            blocks_k, block_indices = self._get_blocks(head_k, self.block_size, self.stride)
            blocks_v, _ = self._get_blocks(head_v, self.block_size, self.stride)

            # Сжимаем блоки с помощью MLP
            compressed_k = self.compress_blocks(blocks_k, self.head_dim)  # (batch_size, num_blocks, head_dim)
            compressed_v = self.compress_blocks(blocks_v, self.head_dim)  # (batch_size, num_blocks, head_dim)

            all_compressed_k.append(compressed_k)
            all_compressed_v.append(compressed_v)
            all_block_indices.append(block_indices)

        # Объединяем результаты для всех голов
        compressed_k = torch.stack(all_compressed_k, dim=1)  # (batch_size, num_heads, num_blocks, head_dim)
        compressed_v = torch.stack(all_compressed_v, dim=1)  # (batch_size, num_heads, num_blocks, head_dim)

        # Для примера берем индексы из первой головы, они одинаковые для всех
        block_indices = all_block_indices[0]
        num_blocks = len(block_indices)

        # Шаг 3: Вычисление внимания между запросами и сжатыми ключами
        # Для каждого запроса мы вычисляем его внимание к сжатым блокам

        # Матрица внимания: (batch_size, num_heads, seq_len, num_blocks)
        attention_scores = torch.matmul(q, compressed_k.transpose(-1, -2)) * self.scale

        if attention_mask is not None:
            # Применяем маску внимания (если она есть)
            attention_scores = attention_scores + attention_mask

        # Нормализация весов с помощью softmax
        attention_probs = F.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)

        # Шаг 4: Взвешенная сумма сжатых значений
        context_layer = torch.matmul(attention_probs, compressed_v)                # (batch_size, num_heads, seq_len, head_dim)

        # Восстановление исходной формы
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()             # (batch_size, seq_len, num_heads, head_dim)
        context_layer = context_layer.view(batch_size, seq_len, self.hidden_size)  # (batch_size, seq_len, hidden_size)

        # Финальная проекция
        output = self.out_proj(context_layer)

        if output_attentions:
            return output, attention_probs, block_indices
        else:
            return output


def demonstrate_compressed_attention(use_long_sequence=False):
    """
    Description:
      Демонстрирует работу механизма сжатого внимания
      Показывает сравнение с обычным полным вниманием

    Аргументы:
        use_long_sequence: если True, использует последовательность длиной 32K токенов
    """
    # Параметры для демонстрации
    hidden_size = 64
    num_heads = 4
    batch_size = 1

    if use_long_sequence:
        # Параметры для длинной последовательности (32K)
        seq_len = 32000
        block_size = 256  # Увеличиваем размер блока для более эффективного сжатия
        stride = 128      # Увеличиваем шаг для более эффективного сжатия
    else:
        # Параметры для короткой последовательности (128)
        seq_len = 128
        block_size = 32
        stride = 16

    # Создаем модели
    compressed_attention = CompressedAttention(
        hidden_size=hidden_size,
        block_size=block_size,
        stride=stride,
        num_heads=num_heads
    )

    # Создаем входные данные с определенными паттернами
    # Мы создадим последовательность, где некоторые токены будут "важными"
    torch.manual_seed(42)  # Для воспроизводимости

    print(f"📌 Создание входных данных длиной {seq_len} токенов...")

    # Базовый входной тензор (создаем эффективно, без чрезмерного использования памяти)
    x = torch.zeros(batch_size, seq_len, hidden_size)

    # Заполняем шумом более эффективно (по частям)
    chunk_size = 1000 if use_long_sequence else seq_len
    for i in range(0, seq_len, chunk_size):
        end = min(i + chunk_size, seq_len)
        x[:, i:end, :] = torch.randn(batch_size, end-i, hidden_size) * 0.1

    # Добавляем "важные" токены через равные промежутки
    # Для длинной последовательности увеличиваем интервал
    important_interval = 1000 if use_long_sequence else 10
    important_positions = list(range(0, seq_len, important_interval))
    for pos in important_positions:
        if pos < seq_len:
            x[:, pos, :] = torch.ones(hidden_size)  # Выделяем важные токены значением 1

    # Добавляем несколько кластеров "важных" токенов
    if use_long_sequence:
        # Создаем 3 кластера в начале, середине и конце последовательности
        cluster_positions = [
            (1000, 1200),      # Начало
            (seq_len//2-100, seq_len//2+100),  # Середина
            (seq_len-1200, seq_len-1000)       # Конец
        ]
    else:
        # Для короткой последовательности - один кластер в середине
        middle_start = seq_len // 3
        cluster_positions = [(middle_start, middle_start + 20)]

    # Добавляем кластеры важных токенов
    for start, end in cluster_positions:
        for pos in range(start, end):
            if pos < seq_len:
                x[:, pos, :] = torch.ones(hidden_size) * 0.8  # Кластер с чуть меньшей важностью

    # Вычисляем важность только для части токенов в длинной последовательности
    token_importance = x.sum(dim=2).squeeze().cpu().numpy()

    print("\n" + "="*80)
    print("ДЕМОНСТРАЦИЯ МЕХАНИЗМА СЖАТОГО ВНИМАНИЯ (COMPRESSED ATTENTION)")
    print("="*80 + "\n")

    print(f"📌 Инициализация модели CompressedAttention с параметрами:")
    print(f"  - Размер скрытого состояния (hidden_size): {hidden_size}")
    print(f"  - Размер блока (block_size): {block_size}")
    print(f"  - Шаг (stride): {stride}")
    print(f"  - Количество голов внимания (num_heads): {num_heads}\n")

    print(f"📌 Создание входных данных:")
    print(f"  - Размер пакета (batch_size): {batch_size}")
    print(f"  - Длина последовательности (seq_len): {seq_len}\n")

    print(f"📌 Подготовка данных:")
    print(f"  - Создали последовательность с несколькими паттернами:")
    if use_long_sequence:
        print(f"    1. Равномерно распределенные 'важные' токены каждые 1000 позиций")
        print(f"    2. Кластеры 'важных' токенов в начале (1000-1200), середине и конце последовательности")
    else:
        print(f"    1. Равномерно распределенные 'важные' токены каждые 10 позиций")
        print(f"    2. Кластер 'важных' токенов в середине (позиции {cluster_positions[0][0]}-{cluster_positions[0][1]})")
    print(f"    3. Случайный шум для остальных токенов\n")

    print(f"📌 Важность токенов (сумма значений по скрытому измерению, примеры):")

    # Для длинной последовательности показываем только образцы
    if use_long_sequence:
        # Показываем начало, середину и конец последовательности
        sample_ranges = [
            (0, 32),                          # Начало
            (seq_len//2-16, seq_len//2+16),   # Середина
            (seq_len-32, seq_len)             # Конец
        ]

        for start, end in sample_ranges:
            print(f"  Позиции {start:5d}-{end-1:5d} (пример):")
            for i in range(start, end, 16):
                end_i = min(i + 16, end)
                values = [f"{token_importance[j]:4.1f}" for j in range(i, end_i)]
                print(f"    {i:5d}-{end_i-1:5d}: {' '.join(values)}")
    else:
        # Для короткой последовательности показываем все токены
        for i in range(0, seq_len, 16):
            end = min(i + 16, seq_len)
            values = [f"{token_importance[j]:4.1f}" for j in range(i, end)]
            print(f"  Позиции {i:3d}-{end-1:3d}: {' '.join(values)}")
    print()

    # Проекции запросов, ключей и значений
    q = compressed_attention.q_proj(x)
    k = compressed_attention.k_proj(x)
    v = compressed_attention.v_proj(x)

    print(f"📌 Шаг 1: Проекции запросов, ключей и значений")
    print(f"  - Форма запросов (q): {q.shape}")
    print(f"  - Форма ключей (k): {k.shape}")
    print(f"  - Форма значений (v): {v.shape}\n")

    print(f"📌 Шаг 2: Сжатие ключей и значений (самая важная часть)")
    print(f"  - Размер блока (block_size): {block_size}")
    print(f"  - Шаг (stride): {stride}")

    # Разделение k на головы внимания
    k_heads = k.view(batch_size, seq_len, num_heads, hidden_size // num_heads).permute(0, 2, 1, 3)
    v_heads = v.view(batch_size, seq_len, num_heads, hidden_size // num_heads).permute(0, 2, 1, 3)

    # Получаем блоки для первой головы
    head_k = k_heads[:, 0]  # (batch_size, seq_len, head_dim)
    blocks_k, block_indices = compressed_attention._get_blocks(head_k, block_size, stride)

    num_blocks = len(blocks_k)
    print(f"  - Количество блоков после разбиения: {num_blocks}")
    print(f"  - Индексы блоков: {block_indices}\n")

    # Сжимаем блоки
    head_dim = hidden_size // num_heads
    compressed_k = compressed_attention.compress_blocks(blocks_k, head_dim)

    print(f"  - Форма сжатых ключей: {compressed_k.shape}")
    print(f"  - Коэффициент сжатия: {seq_len} / {compressed_k.shape[1]} = {seq_len / compressed_k.shape[1]:.1f}x\n")

    # Запускаем полное вычисление
    print(f"📌 Шаг 3: Вычисление внимания на сжатых представлениях")
    output, attention_probs, block_indices = compressed_attention(x, output_attentions=True)

    # Визуализируем матрицу внимания для первой головы
    print(f"  - Форма выхода: {output.shape}")
    print(f"  - Форма матрицы внимания: {attention_probs.shape}\n")

    # Сравниваем вычислительную сложность
    print(f"📌 Шаг 4: Сравнение вычислительной сложности")

    # Стандартное внимание: O(seq_len^2)
    full_attention_complexity = seq_len * seq_len

    # Сжатое внимание: O(seq_len * num_blocks)
    compressed_attention_complexity = seq_len * num_blocks

    print(f"  - Стандартное внимание (Full Attention): O(seq_len^2) = {full_attention_complexity}")
    print(f"  - Сжатое внимание (Compressed Attention): O(seq_len * num_blocks) = {compressed_attention_complexity}")
    print(f"  - Сокращение сложности: {full_attention_complexity / compressed_attention_complexity:.1f}x\n")

    # Измеряем реальное время выполнения
    print(f"📌 Шаг 5: Измерение времени выполнения")

    # Реализация стандартного внимания для сравнения
    def standard_attention(q, k, v, scale):
        attention_scores = torch.matmul(q, k.transpose(-1, -2)) * scale
        attention_probs = F.softmax(attention_scores, dim=-1)
        context_layer = torch.matmul(attention_probs, v)
        return context_layer, attention_probs

    # Делаем запросы, ключи и значения для одной головы
    q_head = q.view(batch_size, seq_len, num_heads, head_dim)[:, :, 0, :]  # (batch_size, seq_len, head_dim)
    k_head = k.view(batch_size, seq_len, num_heads, head_dim)[:, :, 0, :]
    v_head = v.view(batch_size, seq_len, num_heads, head_dim)[:, :, 0, :]

    # Измеряем время для стандартного внимания
    if use_long_sequence:
        print("  - Для полной последовательности 32K токенов стандартное внимание слишком затратно")
        print("  - Оцениваем время на подмножестве данных (1000 токенов) и экстраполируем")

        # Используем только часть данных для оценки времени
        sample_size = 1000
        q_sample = q_head[:, :sample_size, :]
        k_sample = k_head[:, :sample_size, :]
        v_sample = v_head[:, :sample_size, :]

        # Измеряем время на подмножестве
        start_time = time.time()
        for _ in range(10):  # Меньше итераций для длинной последовательности
            _, _ = standard_attention(q_sample, k_sample, v_sample, compressed_attention.scale)
        sample_std_time = (time.time() - start_time) / 10

        # Экстраполируем время для полной последовательности (квадратичная зависимость)
        scaling_factor = (seq_len / sample_size) ** 2
        std_time = sample_std_time * scaling_factor
        print(f"  - Измеренное время для {sample_size} токенов: {sample_std_time:.6f} с")
        print(f"  - Экстраполированное время для {seq_len} токенов: {std_time:.6f} с")
    else:
        # Для короткой последовательности измеряем напрямую
        start_time = time.time()
        for _ in range(100):  # Повторяем несколько раз для более точного измерения
            _, _ = standard_attention(q_head, k_head, v_head, compressed_attention.scale)
        std_time = (time.time() - start_time) / 100

    # Измеряем время для сжатого внимания
    repeat_count = 10 if use_long_sequence else 100  # Меньше итераций для длинной последовательности
    start_time = time.time()
    for _ in range(repeat_count):
        _, _ = compressed_attention(x, output_attentions=True)[:2]
    compressed_time = (time.time() - start_time) / repeat_count

    print(f"  - Стандартное внимание: {std_time:.6f} с")
    print(f"  - Сжатое внимание: {compressed_time:.6f} с")
    print(f"  - Ускорение: {std_time / compressed_time:.2f}x\n")

    # Визуализация
    print(f"📌 Шаг 6: Визуализация матрицы внимания")

    if use_long_sequence:
        print("  - Для 32K токенов визуализируем только фрагмент матрицы внимания")

        # Для длинной последовательности визуализируем только часть матрицы
        # Выбираем интересные фрагменты: начало, середина, конец
        sample_ranges = [
            (0, 500),                           # Начало
            (seq_len//2-250, seq_len//2+250),   # Середина
            (seq_len-500, seq_len)              # Конец
        ]

        fig, axes = plt.subplots(3, 1, figsize=(12, 18))

        for i, (start, end) in enumerate(sample_ranges):
            # Для визуализации используем первую голову внимания и первый пример в батче
            attention_fragment = attention_probs[0, 0, start:end].cpu().detach().numpy()

            # Визуализируем фрагмент матрицы внимания как тепловую карту
            im = axes[i].imshow(attention_fragment, cmap='viridis', aspect='auto')
            fig.colorbar(im, ax=axes[i], label='Вес внимания')

            # Настраиваем оси
            axes[i].set_xlabel('Индекс блока')
            axes[i].set_ylabel(f'Индекс запроса ({start}-{end})')
            axes[i].set_title(f'Фрагмент матрицы внимания ({start}-{end})')

            # Устанавливаем метки тиков
            axes[i].set_xticks(np.arange(len(block_indices)))
            axes[i].set_xticklabels([f"{s}-{e}" for s, e in block_indices], rotation=45, fontsize=8)

            # Показываем только некоторые токены на y-оси для ясности
            fragment_len = end - start
            y_step = max(1, fragment_len // 10)
            y_ticks = np.arange(0, fragment_len, y_step)
            axes[i].set_yticks(y_ticks)
            axes[i].set_yticklabels([str(start + j) for j in y_ticks], fontsize=8)

        # Сохраняем полную матрицу внимания для анализа
        attention_head = attention_probs[0, 0].cpu().detach().numpy()

    else:
        # Для короткой последовательности визуализируем всю матрицу
        # Для визуализации используем первую голову внимания и первый пример в батче
        attention_head = attention_probs[0, 0].cpu().detach().numpy()

        # Создаем сетку позиций токенов
        token_positions = np.arange(seq_len)

        # Преобразуем индексы блоков в средние позиции
        block_positions = [(start + end) // 2 for start, end in block_indices]

        fig, ax = plt.subplots(figsize=(12, 8))

        # Визуализируем матрицу внимания как тепловую карту
        im = ax.imshow(attention_head, cmap='viridis', aspect='auto')
        fig.colorbar(im, ax=ax, label='Вес внимания')

        # Настраиваем оси
        ax.set_xlabel('Индекс блока')
        ax.set_ylabel('Индекс запроса (токена)')
        ax.set_title('Матрица внимания для сжатого внимания (Compressed Attention)')

        # Устанавливаем метки тиков
        ax.set_xticks(np.arange(len(block_indices)))
        ax.set_xticklabels([f"{start}-{end}" for start, end in block_indices], rotation=45)

        # Показываем только некоторые токены на y-оси для ясности
        y_ticks = np.arange(0, seq_len, 16)
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([str(i) for i in y_ticks])

    plt.tight_layout()
    print("  - Матрица внимания визуализирована")

    # Визуализация важных блоков
    print("\n📌 Шаг 7: Анализ сжатых блоков")

    # Находим наиболее важные блоки (по сумме весов внимания)
    if use_long_sequence:
        print("  - Для 32K токенов анализируем агрегированные данные")

        # Для длинной последовательности вычисляем важность блоков как среднее
        # по нескольким ключевым фрагментам для экономии вычислений
        fragment_samples = [
            0,                  # Начало
            seq_len // 4,       # Первая четверть
            seq_len // 2,       # Середина
            3 * seq_len // 4,   # Третья четверть
            seq_len - 1         # Конец
        ]

        # Собираем данные по фрагментам
        fragment_importances = []
        for pos in fragment_samples:
            fragment_row = attention_probs[0, 0, pos].cpu().detach().numpy()
            fragment_importances.append(fragment_row)

        # Усредняем данные по всем фрагментам
        block_importance = np.mean(fragment_importances, axis=0)
    else:
        # Для короткой последовательности используем полную информацию
        block_importance = attention_head.sum(axis=0)

    top_blocks_idx = np.argsort(block_importance)[-3:][::-1]

    print(f"  - Наиболее важные блоки:")
    for i, idx in enumerate(top_blocks_idx):
        start, end = block_indices[idx]
        importance = block_importance[idx]
        print(f"    {i+1}. Блок {idx} (позиции {start}-{end}): важность = {importance:.3f}")

    print("\n📌 Шаг 8: Сравнение с обычным вниманием для заданного запроса")

    # Выбираем определенный запрос для анализа
    if use_long_sequence:
        # Для длинной последовательности выбираем запрос из середины кластера
        query_idx = seq_len // 2

        print(f"  - Анализ внимания для запроса в позиции {query_idx}:")
        print(f"    * В обычном внимании этот запрос распределял бы своё внимание на все {seq_len} токенов")
        print(f"    * В сжатом внимании внимание распределяется только на {len(block_indices)} блоков")

        # Для обычного внимания экстраполяция
        print(f"\n    (Обычное внимание для 32K не вычисляется из-за высокой вычислительной сложности)")

        # Для сжатого внимания
        compressed_attention_probs = attention_probs[0, 0, query_idx].cpu().detach().numpy()

        # Показываем топ блоков для сжатого внимания
        top_k = 5
        top_compressed_idx = np.argsort(compressed_attention_probs)[-top_k:][::-1]
        print(f"\n    Топ-{top_k} блоков в сжатом внимании для запроса {query_idx}:")
        for i, idx in enumerate(top_compressed_idx):
            start, end = block_indices[idx]
            print(f"      {i+1}. Блок {idx} (позиции {start}-{end}): вес = {compressed_attention_probs[idx]:.4f}")
    else:
        # Для короткой последовательности - как в оригинале
        # Выбираем определенный запрос для анализа (например, токен в позиции важного кластера)
        query_idx = cluster_positions[0][0] + 5  # Индекс запроса в середине важного кластера

        # Вычисляем обычное внимание для этого запроса
        q_token = q_head[:, query_idx:query_idx+1, :]  # (batch_size, 1, head_dim)
        attention_scores = torch.matmul(q_token, k_head.transpose(-1, -2)) * compressed_attention.scale  # (batch_size, 1, seq_len)
        full_attention_probs = F.softmax(attention_scores, dim=-1).squeeze().cpu().detach().numpy()

        # Вычисляем сжатое внимание для этого запроса
        compressed_attention_probs = attention_head[query_idx]

        print(f"  - Анализ внимания для запроса в позиции {query_idx}:")
        print(f"    * В обычном внимании этот запрос распределяет своё внимание на все {seq_len} токенов")
        print(f"    * В сжатом внимании внимание распределяется только на {len(block_indices)} блоков")

        # Сравниваем распределение внимания
        top_k = 5  # Показываем топ-5 наиболее важных токенов/блоков

        # Для обычного внимания
        top_tokens_idx = np.argsort(full_attention_probs)[-top_k:][::-1]
        print(f"\n    Топ-{top_k} токенов в обычном внимании:")
        for i, idx in enumerate(top_tokens_idx):
            print(f"      {i+1}. Токен {idx}: вес = {full_attention_probs[idx]:.4f}")

        # Для сжатого внимания
        top_compressed_idx = np.argsort(compressed_attention_probs)[-top_k:][::-1]
        print(f"\n    Топ-{top_k} блоков в сжатом внимании:")
        for i, idx in enumerate(top_compressed_idx):
            start, end = block_indices[idx]
            print(f"      {i+1}. Блок {idx} (позиции {start}-{end}): вес = {compressed_attention_probs[idx]:.4f}")

    print("\n📌 Заключение")
    if use_long_sequence:
        print("  - Механизм сжатого внимания успешно работает с длинной последовательностью (32K токенов)")
        print(f"  - Количество блоков: {len(block_indices)} вместо {seq_len} токенов")
        print(f"  - Коэффициент сжатия: {seq_len / len(block_indices):.1f}x")
        print(f"  - Теоретическое сокращение вычислений: {(seq_len**2) / (seq_len * len(block_indices)):.1f}x")
        print(f"  - Для стандартного внимания потребовалось бы около {std_time:.4f} секунд (экстраполяция)")
        print(f"  - Для сжатого внимания потребовалось {compressed_time:.4f} секунд")
        print(f"  - Теоретическое ускорение: {std_time / compressed_time:.2f}x")
        print("  - Сжатое внимание особенно эффективно для длинных последовательностей")
        print("  - При обработке длинных последовательностей преимущество в скорости многократно перевешивает")
        print("    накладные расходы на сжатие блоков")
    else:
        print("  - Механизм сжатого внимания успешно снижает вычислительную сложность")
        print(f"  - Сокращение вычислений: {full_attention_complexity / compressed_attention_complexity:.1f}x")
        print(f"  - Ускорение: {std_time / compressed_time:.2f}x")
        print("  - При этом сохраняется способность модели фокусироваться на важных частях контекста")
        print("  - Сжатое внимание особенно эффективно для длинных последовательностей")

    return fig

if __name__ == "__main__":
    # По умолчанию запускаем демонстрацию на короткой последовательности
    print("\n=== ДЕМОНСТРАЦИЯ НА КОРОТКОЙ ПОСЛЕДОВАТЕЛЬНОСТИ (128 токенов) ===")
    fig_short = demonstrate_compressed_attention(use_long_sequence=False)

    # Сохраняем изображение
    plt.savefig('compressed_attention_short.png')
    plt.close(fig_short)
    print("\nИзображение сохранено в файл 'compressed_attention_short.png'")

    # Спрашиваем пользователя, хочет ли он запустить тест на длинной последовательности
    run_long_test = input("\nХотите запустить тест на последовательности длиной 32K токенов? (y/n): ")

    if run_long_test.lower() == 'y':
        print("\n=== ДЕМОНСТРАЦИЯ НА ДЛИННОЙ ПОСЛЕДОВАТЕЛЬНОСТИ (32K токенов) ===")
        print("Обратите внимание: этот тест может занять значительное время и потребовать много памяти")

        try:
            # Проверяем доступность GPU и свободную память
            if torch.cuda.is_available():
                free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)
                print(f"Доступная память GPU: {free_memory / 1024**3:.2f} ГБ")

                if free_memory < 4 * 1024**3:  # Меньше 4 ГБ свободной памяти
                    print("Предупреждение: мало свободной памяти на GPU, возможны ошибки OOM")

            # Запускаем тест на длинной последовательности
            fig_long = demonstrate_compressed_attention(use_long_sequence=True)

            # Сохраняем изображение
            plt.savefig('compressed_attention_long.png')
            plt.close(fig_long)
            print("\nИзображение сохранено в файл 'compressed_attention_long.png'")

        except RuntimeError as e:
            print(f"\nПроизошла ошибка: {e}")
            print("Возможно, не хватает памяти для обработки последовательности длиной 32K токенов.")
            print("Попробуйте запустить тест на компьютере с большим объемом памяти или уменьшить длину последовательности.")
    else:
        print("\nТест на длинной последовательности пропущен")


=== ДЕМОНСТРАЦИЯ НА КОРОТКОЙ ПОСЛЕДОВАТЕЛЬНОСТИ (128 токенов) ===
📌 Создание входных данных длиной 128 токенов...

ДЕМОНСТРАЦИЯ МЕХАНИЗМА СЖАТОГО ВНИМАНИЯ (COMPRESSED ATTENTION)

📌 Инициализация модели CompressedAttention с параметрами:
  - Размер скрытого состояния (hidden_size): 64
  - Размер блока (block_size): 32
  - Шаг (stride): 16
  - Количество голов внимания (num_heads): 4

📌 Создание входных данных:
  - Размер пакета (batch_size): 1
  - Длина последовательности (seq_len): 128

📌 Подготовка данных:
  - Создали последовательность с несколькими паттернами:
    1. Равномерно распределенные 'важные' токены каждые 10 позиций
    2. Кластер 'важных' токенов в середине (позиции 42-62)
    3. Случайный шум для остальных токенов

📌 Важность токенов (сумма значений по скрытому измерению, примеры):
  Позиции   0- 15: 64.0  0.8 -0.2  0.7 -0.3  0.6 -0.2 -0.4 -0.0  0.3 64.0  1.2 -1.1 -0.9  0.4 -0.3
  Позиции  16- 31:  0.4  0.6  1.4 -0.8 64.0  0.2 -0.3 -1.3 -0.5 -0.2  0.1 -0.2 -0.5  0.2 64.